<a href="https://colab.research.google.com/github/MattIzon/16010269_DataAnalytics/blob/main/1_Crime_Aquisition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This redirects to get an auth key from your google account. Ctrl + v it into the box and hit enter.
# It sometimes fails on first run then authenticates immediately on second click?

from google.colab import auth
auth.authenticate_user()
print('Authenticated')

from google.cloud import bigquery
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Authenticated
Mounted at /gdrive


In [ ]:
# This uses big query to get the crime data

client = bigquery.Client(project='assignment-305921')

crime = []

for year in range(2016, 2021):
  crime.append(client.query('''
  SELECT *
  FROM `bigquery-public-data.chicago_crime.crime`
  WHERE year = {}
  ORDER BY date
  '''.format(year)).to_dataframe())

In [ ]:
# Save crime data to CSV files.

names = ['bq_2016', 'bq_2017', 'bq_2018', 'bq_2019', 'bq_2020']

for index in range(len(names)):
  with open('/gdrive/My Drive/assignment/data/crime/{}.csv'.format(names[index]), 'w') as f:
    crime[index].to_csv(f)